# 0117-RAGAS-Testset-Generation-for-RAG.ipynb

https://docs.ragas.io/en/latest/getstarted/rag_testset_generation/

### 失敗：RAGAS 自動生成目前有問題

https://github.com/explodinggradients/ragas/issues/1730

## Config

In [1]:
import os
from dotenv import load_dotenv
load_dotenv(".env")

os.environ["LANGCHAIN_TRACING_V2"] = "true"
LANGCHAIN_API_KEY = os.environ["LANGCHAIN_API_KEY"]

Azure_OPENAI_API_KEY = os.environ["AZURE_OPENAI_API_KEY"]

from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper

generator_llm = LangchainLLMWrapper(AzureChatOpenAI(
    api_key=os.environ["MY_AZURE_OPENAI_API_KEY"],
    azure_endpoint = os.environ["MY_AZURE_OPENAI_ENDPOINT"],
    azure_deployment = os.environ["MY_AZURE_OPENAI_DEPLOYMENT_NAME_CHAT"],
    openai_api_version = os.environ["AZURE_OPENAI_API_VERSION"],
    temperature=0
))

# init the embeddings for answer_relevancy, answer_correctness and answer_similarity
generator_embeddings = LangchainEmbeddingsWrapper(AzureOpenAIEmbeddings(
    azure_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"],
    azure_deployment = os.environ["AZURE_OPENAI_DEPLOYMENT_NAME_EMBEDDINGS"],
    openai_api_version = os.environ["AZURE_OPENAI_API_VERSION"],
))

/opt/anaconda3/envs/ragas/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Input doc

In [2]:
from langchain_community.document_loaders import TextLoader, MergedDataLoader

doc_1 = TextLoader("./data/90-Romance-of-the-Three-Kingdoms.txt")
doc_2 = TextLoader("./data/91-Romance-of-the-Three-Kingdoms.txt")
doc_3 = TextLoader("./data/92-Romance-of-the-Three-Kingdoms.txt")

docs = MergedDataLoader(loaders=[doc_1, doc_2, doc_3]).load()

## Generate Testset

In [3]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs, testset_size=10)

Applying SummaryExtractor:  83%|████████▎ | 5/6 [00:03<00:00,  2.57it/s]Property 'summary' already exists in node '8be4c5'. Skipping!
Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/6 [00:00<?, ?it/s]Property 'summary_embedding' already exists in node '8be4c5'. Skipping!
Property 'summary_embedding' already exists in node 'f2ea0a'. Skipping!
Property 'summary_embedding' already exists in node 'e1175e'. Skipping!
Generating Scenarios: 100%|██████████| 1/1 [00:04<00:00,  4.11s/it]
Generating Samples: 0it [00:00, ?it/s]


In [4]:
dataset.to_pandas()

""


## 示意輸出

![img](https://docs.ragas.io/en/latest/getstarted/testset_output.png)